In [68]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

try:
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()
except ImportError:
    import tensorflow as tf

# Common imports
import numpy as np
import os

from tensorflow_graph_in_jupyter import show_graph
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
import tfgraphviz as tfg

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime

In [69]:
print(tf.__version__)

2.1.0


In [70]:
import numpy as np
import os
import tarfile
import joblib
from sklearn.datasets.base import _pkl_filepath, get_data_home

archive_path = "cal_housing.tgz" # change the path if it's not in the current directory
data_home = get_data_home(data_home=None) # change data_home if you are not using ~/scikit_learn_data
if not os.path.exists(data_home):
    os.makedirs(data_home)
filepath = _pkl_filepath(data_home, 'cal_housing.pkz')

with tarfile.open(mode="r:gz", name=archive_path) as f:
    cal_housing = np.loadtxt(
        f.extractfile('CaliforniaHousing/cal_housing.data'),
        delimiter=',')
    # Columns are not in the same order compared to the previous
    # URL resource on lib.stat.cmu.edu
    columns_index = [8, 7, 2, 3, 4, 5, 6, 1, 0]
    cal_housing = cal_housing[:, columns_index]

    joblib.dump(cal_housing, filepath, compress=6)

In [71]:
housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [72]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def createLogDir(root_logdir="tf_logs"):
    now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
    logdir = "{}/run-{}/".format(root_logdir, now)
    return logdir

In [73]:
n_epochs = 2000
learning_rate = 0.01

logDir = createLogDir(root_logdir="./tf_logs")

print(logDir)

./tf_logs/run-20201111233338/


In [74]:
reset_graph()

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logDir, tf.get_default_graph())

In [75]:
with tf.Session() as sess:
    
    if os.path.isfile(logDir+'my_model_final.ckpt.meta'):
        print('File exists.')
        saver.restore(sess, logDir+"my_model_final.ckpt")
    else:
        print('File does not exist.')
        sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
            save_path = saver.save(sess, logDir+"my_model.ckpt")

        summary_str = mse_summary.eval()
        file_writer.add_summary(summary_str, epoch)

        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, logDir+"my_model_final.ckpt")
    
file_writer.close()

File does not exist.
Epoch 0 MSE = 9.16154
Epoch 100 MSE = 0.7145003
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.55557173
Epoch 400 MSE = 0.5488112
Epoch 500 MSE = 0.54363626
Epoch 600 MSE = 0.53962904
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473
Epoch 1000 MSE = 0.53062934
Epoch 1100 MSE = 0.52942395
Epoch 1200 MSE = 0.52846223
Epoch 1300 MSE = 0.5276917
Epoch 1400 MSE = 0.527072
Epoch 1500 MSE = 0.52657175
Epoch 1600 MSE = 0.5261665
Epoch 1700 MSE = 0.52583706
Epoch 1800 MSE = 0.5255686
Epoch 1900 MSE = 0.5253494


In [1]:
!kill 16144

'kill' não é reconhecido como um comando interno
ou externo, um programa operável ou um arquivo em lotes.


In [2]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard

In [3]:
%tensorboard --logdir './tf_logs'

Reusing TensorBoard on port 6006 (pid 16144), started 4 days, 13:32:53 ago. (Use '!kill 16144' to kill it.)